# Running Auto Correlation with Talon Deployer and BITE
###### Last Updated: 12/04/24

This demo will show the basic operation of auto correlation using both the original docker-based Deployer and BITE commands, as well as the new TANGO device based ones. With this notebook, all TANGO commands and attribute changes are made via a [TANGO DeviceProxy](https://pytango.readthedocs.io/en/stable/client_api/device_proxy.html) but the overall steps should be the same for using the JIVE interface or Taranta web interface.

## 1 Prerequisites


First, this notebook expects a running environment launched from a pipeline, in particular it assumes one launched from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline. Secondly, for ease of dev work, it also assumes it is run in a virtual env. This notebook was made with Python 3.10 in mind.

Finally, make sure all requirements are installed via [poetry](https://python-poetry.org/docs/basic-usage/#installing-dependencies), and after that, grab the imports required.

In [ ]:
import json
import os
import sys
from datetime import datetime
from time import sleep

from astropy.time import Time

sys.path.append("../../src")
from tango import Database, DevFailed, DeviceProxy

import notebook_tools.wait_for_tango as wait_for_tango

### 1.1 Running on PSI


To run on PSI, once running the launch step from the PSI pipeline, grab the booted namespace's name. Also run a check on the ns as well to make sure the pods are ready.

In [ ]:
!kubectl get ns | grep ska-mid-psi

In [ ]:
!kubectl -n ci-ska-mid-psi-1280777666-jaredmda get pods

### 1.2 Loading in Variables/Checking Taranta

Now, set the variables we'll need for the run. First the constants that should not need to be changed unless debugging:

In [ ]:
# --Requried Vars--
# Non-SPD namespace
ns = ""
# The board to be used for the test.
target_talon = 0
# Set to 0 if to run on actual hardware, 1 to simulate
sim_mode = 0

# --Optional vars--
# Change these values if to specify a certain slim directory for files
slim_fs_config = ""
slim_vis_config = ""

In [ ]:
# MAC address for the boards.
TARGET_MAC_ADDRESS = "08:c0:eb:9d:47:78"
# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIGS = os.path.join(DATA_DIR, "mid_telescope/cbf")
CSP_CONFIGS = os.path.join(DATA_DIR, "mid_telescope/csp")
TMC_CONFIGS = os.path.join(DATA_DIR, "mid_telescope/tmc")
HW_CONFIGS = os.path.join(COMMON_CONFIGS, "hw_config")
# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]

With these set, check the taranta dash to monitor the boards as the rest of the notebook is run through.

In [ ]:
url = "https://142.73.34.170/" + ns + "/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run"
print(
    "Monitor board status using: https://142.73.34.170/{}/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run".format(
        ns
    )
)

At this point, all components should be in the `disabled` state.

## 2 Initial Setup

First, to set up the target board that that will be used for the auto correlation. While this can be multiple boards, for now only one is needed. Punch out the board(s) and set the `target_talon` to assign it to be used in future steps.

The `TANGO_HOST` will be created based of the namespace set earlier.

In [ ]:
TANGO_HOST = "databaseds-tango-base." + ns + ".svc.cluster.local:10000"
print("Will be using HOST: ", TANGO_HOST)
os.environ["TANGO_HOST"] = TANGO_HOST

Now load in the locations of local JSON files for configuration:

In [ ]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIGS, "sys_params/initial_system_param.json")
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIGS, "assign_resources.json")
ASSIGN_CSP_RESOURCES_FILE = os.path.join(CSP_CONFIGS, "assign_resources.json")
CONFIGURE_SCAN_FILE = os.path.join(TMC_CONFIGS, "configure_scan.json")
SCAN_FILE = os.path.join(TMC_CONFIGS, "scan.json")
CSP_DELAY_MODEL_FILE = os.path.join(TMC_CONFIGS, "delay_model.json")

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    CSP_DELAY_MODEL_FILE,
]
print("Checking to make sure files exist...")
for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")


print("Checking done")

Next, pass in the SLIM mesh config files by copying them to the namespace, these files should be in the json_files storage folder. Custom files here are not required, but if needed, the following two code blocks can be used.

In [ ]:
if slim_fs_config != "":
    !"kubectl cp {fs_config_path} {ns}/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml"
    print("Loaded custom SLIM fs config.")
else:
    print("SLIM will use default fs config")
if slim_vis_config != "":
    !"kubectl cp {vis_config_path} {ns}/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml"
    print("Loaded custom SLIM vis config.")
else:
    print("SLIM will use default vis config.")

This demo notebook will interact with the TANGO devices via a device proxy, which will allow us to pass commands into them like the UI.

In [ ]:
# Setup the device proxies targeting bite and deployer
db = Database()
deployer = DeviceProxy("mid_csp_cbf/ec/deployer")
# Check the devices initially deployed to the database
print("Check to ensure database contains expected devices.")
print(*db.get_device_exported("*").value_string, sep="\n")

Next load in the hardware configuration depending on the talon boards selected. If a higher number board is chosen, need to use the swapped config file, and then modify the board value to match the swap file.

In [ ]:
if target_talon > 4:
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_talon = target_talon - 4
else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIGS, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

Now copy this file into the controller pod:

In [ ]:
!kubectl cp $HW_CONFIG_FILE $ns/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

## 3 Deploying Devices

For deploying the required Tango Devices, there are two options:

### 3.1 Deploying Using the Command Line (Old Method)

To use the old method for running the deployer commands, use kubectl's exec function to run commands on the relevant pod, using the deployer script and passing the relevant commands to it as arguments.

In [ ]:
print("Deploying to Talon board -> {}".format(target_talon))
!kubectl exec -ti -n $ns ec-deployer -- python3 midcbf_deployer.py --generate-talondx-config --boards=$target_talon
!kubectl exec -ti -n $ns ec-deployer -- python3 midcbf_deployer.py --download-artifacts
!kubectl exec -ti -n $ns ec-deployer -- python3 midcbf_deployer.py --config-db

Ensure that the download steps runs, and reaches completion.

### 3.2 Deploying Using the Deployer Device

First, set the target talon boards to set up the configuration for. In Jive/Taranta, this would be configured by manually writing the attribute via the UI. Multiple boards can be targeted. With this set, then run the configuration command by calling the generate_config_jsons command.

In [ ]:
deployer.targetTalons = target_talon
print(f"Set deployer device to target boards: {deployer.targetTalons}")
print("Generating configurations for specified board...")
deployer.generate_config_jsons()

Then get the device artifacts from the [artifact repository](https://artefact.skao.int/#browse/browse:helm-internal) by running the command via TANGO. This step may take some time as it downloads multiple devices.

In [ ]:
deployer.set_timeout_millis(200000)
try:
    print("Starting download of required artifacts...")
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(4500)

To check that the artifacts downloaded successfully, check that the following returns something like `INFO|Dummy-2|download_fpga_bitstreams|midcbf_deployer.py#418||Finished downloading`. 

In [ ]:
print("Confirm that this returns that artifacts finished downloading:")
!kubectl logs -n $ns ds-deployer-deployer-0 | grep 'Finished downloading'

Finally, configure the TANGO database with all the tango devices downloaded using the ConfigDB command.

In [ ]:
deployer.configure_db()
print("Confirm that the correct TANGO Devices were exported:")
print(*db.get_device_exported("*").value_string, sep="\n")

With the devices we'll need to use now exported, proceed to use them for the next steps of starting up the boards and running BITE commands.

## 4 Uploading Controller Settings and Starting the Boards

Now that the required devices are deployed and exported, set up the DeviceProxies for the devices to use in the next steps:

In [ ]:
print("Starting  proxies to required devices...")
controller = DeviceProxy("mid-csp/control/0")
print(controller.status())
subarray = DeviceProxy("mid-csp/subarray/01")
print(subarray.status())
cbf = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(cbf.status())
print("Done!")

**Before running this step and the following steps, ensure that the boards to use in this notebook are checked out!**
***

Then set adminMode to 0 (ONLINE), allowing us to run commands, and set simulationMode to 0 (FALSE).

In [ ]:
# Set relevant values on the mid-csp controller
print("Setting admin mode...")
controller.adminMode = 0
sleep(1)
print("Setting simulation mode...")
controller.write_attribute("cbfSimulationMode", sim_mode)
controller.cbfSimulationMode = sim_mode
sleep(1)
if (
    controller.read_attribute("adminMode").value == 0
    and controller.read_attribute("cbfSimulationMode").value == 0
):
    print("Turned on devices, simulation mode set to FALSE!")
elif (
    controller.read_attribute("adminMode").value == 0
    and controller.read_attribute("cbfSimulationMode").value == 1
):
    print("Turned on devices, simulation mode set to TRUE.")
else:
    print("Error, couldn't turn on the device!")

Checking the status dashboard, it should now display that all devices are both OFF and that the `simulationstate` is set to the expected state.

 Next, load in a initial values parameters and pass it to the controller, to do this, read in JSON file and pass it as a DevString to the relevant device command: 

In [ ]:
with open(INIT_SYS_PARAM_FILE, encoding="utf-8") as init_file:
    data = json.load(init_file)
cbf.InitSysParam(json.dumps(data))
print("Uploaded Initial system parameters:")
print(cbf.sysParam)

Now, turn ON the Controller by passing it the device to turn on, and letting it run for 55ms to give the boards time to power on.

In [ ]:
controller.set_timeout_millis(50000)
target = ["mid_csp_cbf/sub_elt/controller"]
print("Sending the command to turn on the selected board and waiting for it to start up...")
controller.On(target)
wait_for_tango.wait_for_status(controller, "ON")

In [ ]:
print(controller.status())

After running this step, check with the Taranta dashboard to check that the boards are started.

## 5 Running BITE Commands

### 5.1 Running Commands through the BITE Device

Now that the BITE tango Device has been deployed via the deployer, use it to configure tests. First, check the device is running.

In [ ]:
bite = DeviceProxy("mid_csp_cbf/ec/bite")
print(f"Current BITE device status (Should be RUNNING): {bite.State()}")

For now, use the defaults and simply call the write command for the test configs. This should return the configuration for each board passed in.

In [ ]:
print("Setting target board for data generation...")
bite.boards = target_talon
bite.bite_mac_address = TARGET_MAC_ADDRESS
bite_mode = "device"
print("Generating bite data...")
bite.command_inout("generate_bite_data")

## 6 Assigning Resources

Next, use the relevant subarray device to assign resources. First, as with the other devices, establish a DeviceProxy to connect to it. Also read in the assign_resources file to load in using the command. 

For the loaded data, ensure the right board is selected, modifying the data for the receptor ID based on the Talon board selected.

In [ ]:
# Load in the data from the file.
subarray = DeviceProxy("mid-csp/subarray/01")
with open(ASSIGN_CSP_RESOURCES_FILE, encoding="utf-8") as init_file:
    config_dict = json.load(init_file)

# In order to use the correct receptor, modify the assign_resources data to use the correct receptor based on the board we're using
config_dict["dish"]["receptor_ids"] = [RECEPTOR_MAP[target_talon - 1]]
print("Modified config data:")
print(json.dumps(config_dict, indent=1))

Next, run the actual command to pass in the resources:

In [ ]:
print("Assigning CSP resources:")
subarray.AssignResources(json.dumps(config_dict))
wait_for_tango.wait_for_state(subarray, "IDLE")

## 7 SDP Setup

Next, run a few commands to set up the Science Data Processor (SDP), utilizing the sdp namespace

First, set up a proxy to the SDP device:

In [ ]:
sdp_subarray = DeviceProxy("mid-sdp/subarray/01")
sdp_subarray.On()
print(f"SDP device status (should be RUNNING): {sdp_subarray.Status()}")

Now, assign resources as with the subarray. As with the csp, map the receptor boards based on our checked out board.

In [ ]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    sdp_resources = json.load(f)["sdp"]
sdp_resources["resources"]["receptors"] = [RECEPTOR_MAP[target_talon - 1]]
print("modified sdp_resources file...")
print(json.dumps(sdp_resources, indent=1))

And run the command to assign resources and wait to get out of the resourcing state:

In [ ]:
print("Assigning resources to SDP...")
sdp_subarray.AssignResources(json.dumps(sdp_resources))
wait_for_tango.wait_for_state(sdp_subarray, "IDLE")

Next send the configure command along with the required config file:

In [ ]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    sdp_configuration = json.load(f)
    sdp_configuration = sdp_configuration["sdp"]
print("SDP scan configuration for this run will be...")
print(json.dumps(sdp_configuration, indent=1))

In [ ]:
print("Configuring SDP...")
sdp_subarray.Configure(json.dumps(sdp_configuration))
wait_for_tango.wait_for_state(sdp_subarray, "READY")

Start the SDP output via the Scan command:

In [ ]:
with open(SCAN_FILE, encoding="utf-8") as f:
    sdp_scan = json.load(f)
    sdp_scan = sdp_scan["sdp"]
print("SDP scan config:")
print(json.dumps(sdp_scan, indent=1))

In [ ]:
print("Starting SDP scan...")
sdp_subarray.Scan(json.dumps(sdp_scan))
print(sdp_subarray.obsState.name)

After running this step, a pod for receiving visibilities will be launched, grab the IP of this pod and place it in the output_host var in the configure_scan json file.

In [ ]:
# Grab the pod name
!kubectl -n $ns-sdp get pods | grep vis-receive

In [ ]:
# Then find the IP:
vis_pod = "proc-pb-test-20211111-00059-vis-receive-00-0"
!kubectl -n $ns-sdp describe pod $vis_pod | grep net1 -A 3

Grab the IP from this, and write it to a var we'll use later to configure the CSP.

In [ ]:
# If using visibility pod, please copy this to the vis_cfg.json
output_host = "10.50.1.30"

With this pod, prep for monitoring the pod for when it receives the visibilities

Run this command, then use the output in a separate terminal to enter the correct pod:

In [ ]:
!echo kubectl exec -n $ns-sdp -ti $vis_pod -- bash

Now run the following commands in this pod to start monitoring for the correct traffic:

## 8 Starting up the CSP Scan

Now on the CSP side, load in the corresponding Scan and config files, modifying the config to prep for our test.

In [ ]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    csp_config = json.load(f)["csp"]
with open(SCAN_FILE, encoding="utf-8") as f:
    csp_scan = json.load(f)["csp"]
# Modify FSP to match selected board
csp_config["cbf"]["fsp"][0]["fsp_id"] = target_talon

# Write to the JSON to match original config scan file
csp_config["common"]["config_id"] = "1 receptor, band 1, 1 FSP, no options"
csp_config["cbf"]["fsp"][0]["fsp_id"] = 1
csp_config["cbf"]["fsp"][0]["zoom_factor"] = 1
csp_config["cbf"]["fsp"][0]["zoom_window_tuning"] = 450000
csp_config["cbf"]["fsp"][0]["channel_offset"] = 0
csp_config["cbf"]["fsp"][0]["receptors"] = [RECEPTOR_MAP[target_talon - 1]]
csp_config["cbf"]["fsp"][0]["output_host"][0][0] = 0
csp_config["cbf"]["fsp"][0]["output_host"][0][1] = output_host

csp_config["cbf"]["fsp"][0]["output_port"][0][0] = 0
csp_config["cbf"]["fsp"][0]["output_port"][0][1] = 21000
csp_config["cbf"]["fsp"][0]["output_port"][0][2] = 1
print("CSP configuration:")
print(json.dumps(csp_config, indent=1))
print("==============================")
print("CSP scan configuration:")
print(json.dumps(csp_scan, indent=1))

In [ ]:
print("Observation state: {}".format(subarray.obsState.name))
subarray.Configure(json.dumps(csp_config))
wait_for_tango.wait_for_state(subarray, "READY")

In [ ]:
print("Observation state: {}".format(subarray.obsState.name))

Then start LSTV replay, check the logs and get the epoch value to use later (`INFO: start_utc_time_offset = start_utc_time.unix_tai - ska_epoch_tai = <EPOCH VALUE TO COPY>`)

In [ ]:
if bite_mode == "device":
    print("Using the bite device to start lstv replay...")
    bite.start_lstv_replay()
else:
    print("Using the command line function to start lstv replay...")
    !kubectl exec -ti -n $ns ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards=$standin_board --input_data=$target_talon

Now store this epoch value to configure the delay model.

In [ ]:
target_epoch = 768350073.0

For the SPFRx, run this code:

In [ ]:
SKA_EPOCH = "1999-12-31T23:59:28Z"
ska_epoch_utc = Time(SKA_EPOCH, scale="utc")
ska_epoch_tai = ska_epoch_utc.unix_tai

start_utc_time = Time(datetime.utcnow(), scale="utc")
target_epoch = start_utc_time.unix_tai - ska_epoch_tai

Then load this into the delay model, and change the `epoch` value to the one generated by the BITE LSTV replay start command, and the `receptor` to match our LSTV gen and receptor ID:

In [ ]:
with open(CSP_DELAY_MODEL_FILE, encoding="utf-8") as f:
    delay_model = json.load(f)
delayModelProxy = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
delay_model["receptor_delays"][0]["receptor"] = RECEPTOR_MAP[target_talon - 1]
delay_model["start_validity_sec"] = target_epoch
print(json.dumps(delay_model))

In [ ]:
print("Writing delay model...")
delayModelProxy = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
delayModelProxy.write_attribute("delayModel", json.dumps(delay_model))

And then the matching command in the subarray:

In [ ]:
print("Starting CSP scan...")
subarray.Scan(json.dumps(csp_scan))
print("Observation state (Should be SCANNING): {}".format(subarray.obsState.name))

Checking the dashboard, the dish should now be in the SCANNING status.

## 9 Checking Visabilties

With the devices scanning and linked, monitor the output via monitoring the network packets that come from it. Use the tcpdump running terminal to check that packet lengths are correct:
- UDP, length 136.
- UDP, length 740.

Also check that the outputs are properly stored.

## 10 Cleanup

Once satisfied with the results, stop the scans and shut down the boards.

In [ ]:
sdp_subarray.EndScan()
subarray.EndScan()

In [ ]:
subarray.GoToIdle()
sleep(10)
subarray.ReleaseAllResources()

In [ ]:
controller.Off(target)

If the boards are on, use the LRU command to turn them off.

Now that we're done, free up dev resources on PSI by deleting the ns.

In [ ]:
!kubectl delete ns $ns
!kubectl delete ns $ns-sdp
!kubectl get ns | grep ska-mid-psi

🎉 Congrats, you've now run the Auto Correlation demo!